In [20]:
import pandas as pd
import numpy as np
from gensim.models.keyedvectors import load_word2vec_format
from sklearn.metrics.pairwise import linear_kernel

In [17]:
from ipynb.fs.full.preprocess_and_generate_dataset import generate_dataframe

In [18]:
class_dataframe = generate_dataframe(10)
class_dataframe = class_dataframe.rename(columns={'Post': 'text', 'Label':'label'})


In [55]:
class_dataframe

,text,label,code
0,viable option youll leaving wife behind youd p...,Supportive,4
1,hard appreciate notion could meet someone else...,Ideation,2
2,hi last night wa sitting ledge window contempl...,Behavior,1
3,tried kill self failed badly cause moment want...,Attempt,0
4,hi nem3030 sort thing enjoy personally always ...,Ideation,2
...,...,...,...
15,Neoplasm of uncertain behavior of central ner...,Indicator,3
5,Suicide or self-injury by arson Suicide and s...,Attempt,0
51,Malignant neoplasm Secondary malignant neopla...,Indicator,3
87,Malignant melanoma Primary malignant neoplasm...,Indicator,3


In [8]:
model_file = '/home/ifte-home/work/resources/w2v_300/model.bin'
dimention = 300

In [7]:
model = load_word2vec_format(model_file, binary=True, unicode_errors='ignore')

In [56]:
df = pd.read_csv('/home/ifte-home/Documents/mental_health/suicide/CSSRS/TFIDF_result_suicide_intensity.csv', index_col=0)

In [57]:
df

,text,label
69567,time read format wanted make book could done j...,Indicator
5571,depressed year story hope help raise awareness...,Indicator
34978,long text friend doubt anyone read tldr highly...,Indicator
28886,fuck idek name call badger year canadian dont ...,Indicator
87420,pickle pickle existence girl matter born write...,Indicator
...,...,...
41618,hentai helped become simp warning loooong text...,Supportive
31865,anyone need reason live would miss worth alive...,Supportive
14780,stressing life frankly sick tired world someti...,Supportive
40249,last advice offer make mistake often severe ot...,Supportive


In [14]:
def get_vector(text, dimention):
    sum_vec = np.zeros(dimention)
    word_count = 0
    for items in text.split():
        try: 
            temp = model[items]
        except KeyError:
            temp = 0
        sum_vec += temp
        word_count += 1
    return sum_vec / word_count


In [58]:
def train_dataset(dataset, dim):
    vector_collection = []
    for i, col in dataset.iterrows():
        vector_collection.append(get_vector(col['text'], dim))
    return np.array(vector_collection)

vector_collection_twitter = train_dataset(df, dimention)

In [60]:
def get_avg_simiarity(search_string, dimention, twitter_vector):
    reddit_vector = get_vector(search_string, dimention).reshape(1,-1)
    rank = linear_kernel(reddit_vector, twitter_vector)
    cosine_similarities = rank.flatten()
    related_docs = cosine_similarities.argsort()[:-500:-1]    
#     print(collect, related_docs)
    output = cosine_similarities[related_docs]
    avg_result = np.sum(output)/related_docs.size
    return avg_result, related_docs[1:500]

In [49]:
CSSR_CAT = ['Indicator', 'Attempt','Behavior','Ideation', 'Supportive']
output = dict.fromkeys(CSSR_CAT)

In [61]:
best_df = pd.DataFrame()
for i in CSSR_CAT:
    _df = class_dataframe[class_dataframe['label']==i]
    _df.reset_index(inplace=True, drop=True)
    text_compare = ' '.join(_df['text'].values.tolist())
    
    temp, best_indices = get_avg_simiarity(text_compare, dimention, vector_collection_twitter)
    output[i] = temp
    
    new_df = df.iloc[best_indices]
    new_df['label_w2v'] = i
    best_df = pd.concat([best_df, new_df])
    

/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on 

In [62]:
output

{'Indicator': 0.20394809028217803,
 'Attempt': 0.20669614199258618,
 'Behavior': 0.21261226336473782,
 'Ideation': 0.21255234277562507,
 'Supportive': 0.21957425838273859}

In [63]:
best_df

,text,label,label_w2v
42035,everyone year quite shit everyone people lost ...,Attempt,Indicator
42035,everyone year quite shit everyone people lost ...,Ideation,Indicator
42035,everyone year quite shit everyone people lost ...,Indicator,Indicator
42035,everyone year quite shit everyone people lost ...,Behavior,Indicator
42035,everyone year quite shit everyone people lost ...,Supportive,Indicator
...,...,...,...
71531,people understand clout farming okay fortniteb...,Indicator,Supportive
109664,youre woogie woogie woogie,Supportive,Supportive
109664,youre woogie woogie woogie,Indicator,Supportive
100803,makeing minecraft skin price negotiable making...,Indicator,Supportive
